In [1]:
import sys
import re
import numpy as np
import pandas as pd
import lightgbm as lgb
import xgboost as xgb

sys.path.append("../")

In [31]:
from lib.preprocess import data_preparation
df, train_size, test_size = data_preparation()

air_store dataframe shape: (829, 57)


In [32]:
TARGET = 'visitors'

visit_num_vars = ['dow', 'year', 'month', 'doy', 'dom', 'woy', 'is_month_end', 'date_int']

reserve_num_vars = ['reserve_ppl_count_air', 'reserve_tot_count_air', 'avg_reserve_hr_day_air',
    'max_reserve_hr_air', 'min_reserve_hr_air', 'mean_reserve_hr_air', 'min_reserve_dy_air',
    'mean_reserve_dy_air', 'max_reserve_dy_air', 'reserve_ppl_count_hpg', 'reserve_tot_count_hpg',
    'avg_reserve_hr_day_hpg', 'max_reserve_hr_hpg', 'min_reserve_hr_hpg', 'mean_reserve_hr_hpg',
     'min_reserve_dy_hpg', 'mean_reserve_dy_hpg', 'max_reserve_dy_hpg']

store_num_vars = [
    'latitude_air', 'longitude_air', 'air_stores_on_same_addr', 'air_stores_lv1',
    'air_stores_lv2', 'air_stores_lv3', 'mean_lat_air_lv1', 'max_lat_air_lv1',
    'min_lat_air_lv1', 'mean_lon_air_lv1', 'max_lon_air_lv1', 'min_lon_air_lv1',
    'mean_lat_air_lv2',  'max_lat_air_lv2', 'min_lat_air_lv2', 'mean_lon_air_lv2',
    'max_lon_air_lv2',   'min_lon_air_lv2', 'air_genre_count', 'air_genre_count_lv1',
    'air_genre_count_lv2',   'air_genre_count_lv3', 'latitude_hpg', 'longitude_hpg',
    'hpg_stores_on_same_addr', 'hpg_stores_lv1', 'hpg_stores_lv2', 'hpg_stores_lv3',
    'mean_lat_hpg_lv1', 'max_lat_hpg_lv1', 'min_lat_hpg_lv1', 'mean_lon_hpg_lv1',
    'max_lon_hpg_lv1', 'min_lon_hpg_lv1', 'mean_lat_hpg_lv2', 'max_lat_hpg_lv2',
    'min_lat_hpg_lv2', 'mean_lon_hpg_lv2', 'max_lon_hpg_lv2', 'min_lon_hpg_lv2',
    'hpg_genre_count', 'hpg_genre_count_lv1', 'hpg_genre_count_lv2', 'hpg_genre_count_lv3'
    ]
store_cat_vars = [
    'air_genre_name', 'air_lv1', 'air_lv2', 'air_lv3', 'air_lv4',
    'hpg_genre_name', 'hpg_lv1', 'hpg_lv2', 'hpg_lv3'
]

interacts_vars = [
    'reserve_ppl_count', 'reserve_tot_count', 'reserve_ppl_mean', 'lon_plus_lat_air',
    'lat_to_mean_lat_air_lv1', 'lat_to_max_lat_air_lv1', 'lat_to_min_lat_air_lv1',
    'lon_to_mean_lon_air_lv1', 'lon_to_max_lon_air_lv1', 'lon_to_min_lon_air_lv1',
    'lat_to_mean_lat_air_lv2', 'lat_to_max_lat_air_lv2', 'lat_to_min_lat_air_lv2',
    'lon_to_mean_lon_air_lv2', 'lon_to_max_lon_air_lv2', 'lon_to_min_lon_air_lv2',
    'lat_to_mean_lat_hpg_lv1', 'lat_to_max_lat_hpg_lv1', 'lat_to_min_lat_hpg_lv1',
    'lon_to_mean_lon_hpg_lv1', 'lon_to_max_lon_hpg_lv1', 'lon_to_min_lon_hpg_lv1',
    'lat_to_mean_lat_hpg_lv2', 'lat_to_max_lat_hpg_lv2', 'lat_to_min_lat_hpg_lv2',
    'lon_to_mean_lon_hpg_lv2', 'lon_to_max_lon_hpg_lv2', 'lon_to_min_lon_hpg_lv2',  
]

hol_mix_vars = ['weight1', 'weight2', 'holiday_flg']

target_agg_vars = ['mean_visitors', 'max_visitors', 'min_visitors', 'median_visitors', 'wmean_visitors']




features = visit_num_vars + reserve_num_vars + store_num_vars + store_cat_vars + interacts_vars + hol_mix_vars + target_agg_vars

train = df[:train_size]
test = df[train_size:]
y = df[:train_size][TARGET][:train_size].values
IDs = df[train_size:][train_size:].id.values
print ('train data size:', train.shape, 'test data size:', test.shape)    

train data size: (252108, 123) test data size: (32019, 123)


In [51]:
train_x = train[train['visit_date'].between('2016-04-01', '2017-03-09')][features].values
train_y = np.log1p(train[train['visit_date'].between('2016-04-01', '2017-03-09')][TARGET].values)

valid_x = train[train['visit_date'] > '2017-03-09'][features].values
valid_y = np.log1p(train[train['visit_date'] > '2017-03-09'][TARGET].values)
print (train_x.shape, valid_x.shape)

(197315, 115) (31591, 115)


In [52]:
xgtrain = xgb.DMatrix(train_x, label=train_y)
xgval=xgb.DMatrix(valid_x,label=valid_y)

watchlist  = [ (xgtrain,'train'),(xgval,'eval')]


best_xgb_params = {'colsample_bytree': 0.7,
 'eta': 0.1,
 'gamma': 1,
 'max_depth': 10,
 'min_child_weight': 3,
 'nthread': 8,
 'objective': 'reg:linear',
 'seed': 1,
 'subsample': 1}

print (best_xgb_params)

model = xgb.train(best_xgb_params, 
                  xgtrain, 
                  num_boost_round=100000,
                  evals=watchlist,
                  early_stopping_rounds=50,
                  verbose_eval=100)    
best_iteration = model.best_iteration
best_score = model.best_score
print ('best_score: %f, best_iteration: %d' % (best_score, best_iteration))

{'colsample_bytree': 0.7, 'eta': 0.1, 'gamma': 1, 'max_depth': 10, 'min_child_weight': 3, 'nthread': 8, 'objective': 'reg:linear', 'seed': 1, 'subsample': 1}
[0]	train-rmse:2.19653	eval-rmse:2.25204
Multiple eval metrics have been passed: 'eval-rmse' will be used for early stopping.

Will train until eval-rmse hasn't improved in 50 rounds.
[100]	train-rmse:0.440243	eval-rmse:0.484031
[200]	train-rmse:0.431116	eval-rmse:0.483825
Stopping. Best iteration:
[179]	train-rmse:0.433678	eval-rmse:0.483671

best_score: 0.483671, best_iteration: 179


In [53]:
print(1)

1


In [54]:
model = xgb.train(best_xgb_params, 
                  xgb.DMatrix(train_x, label=train_y), 
                  num_boost_round=best_iteration)  
sub = test[['id','visitors']].copy()
sub['visitors'] = model.predict(xgb.DMatrix(test[features].values))
sub['visitors'] = np.expm1(sub['visitors']).clip(lower=0.)
sub[['id', 'visitors']].to_csv('../submission/sub_starter.csv', index=False)

I:\Anaconda2\envs\py36\lib\site-packages\ipykernel_launcher.py:4: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  after removing the cwd from sys.path.
I:\Anaconda2\envs\py36\lib\site-packages\ipykernel_launcher.py:5: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  """
